# Assemble the final Dataframe from tarpmir bindingsites

## Read in bindingsite & Ensembl data

In [1]:
%load_ext autoreload
%autoreload 2
from helper_fcts import *
import sqlite3
import os
import pandas as pd
from pathlib import Path
import numpy as np
test = False
path = Path('data')
tcga_path = Path(path/'KICH')
path_tarp = Path(path/'tarp-bs') if test else Path('../data')
path_input = Path(path/'tarp-bs/cdna') if test else Path('../data/all_cdna_subsets')

In [3]:
#test bindingsite
bs = parse_tarp_bs(path/'tarp-bs'/'input_miRNA.fasta_cdna6247.fasta.bp')

In [9]:
#read in df_gene: gene_id, g_id_v, transcript_id, t_id_v, chromosome, strand, start, end (1-N)
df_gene = pd.read_csv(path/'gene_infos.csv', dtype={'ensembl_gene_id':str,'ensembl_gene_id_version':str,'ensembl_transcript_id':str, 'ensembl_transcript_id_version':str, 'chromosome_name':'category', 'strand':'int8', 'start_position':int, 'end_position':int})
del df_gene['Unnamed: 0']
df_gene[['strand']] = df_gene[['strand']].astype('str').astype('category')

#executed once, read in from pickled files
exon_positions = read_in_exon_pos(path/'exon_positions.csv') # mapping of transcript ids to exon ids, chrom_exon_starts, exon_starts, exon_ends

#df_exon = calc_exon_data(path) # exon starts and ends per transcript + exon id
#df_exon = df_exon.merge(df_gene, left_on = 'transcript_id', right_on = 'ensembl_transcript_id', how='left')
#df_exon[['chrom_exon_start','chrom_exon_end','start_position','end_position']] = df_exon[['chrom_exon_start','chrom_exon_end','start_position','end_position']].astype('Int64')
#df_exon[['strand']] = df_exon[['strand']].astype('str').astype('category')

#TODO put overall more function in helper

#len(df_exon[df_exon.ensembl_transcript_id.isna()].transcript_id.unique())
#371 transcripts are not in df_gene, TODO, why? which transcripts are in tarpmir predictions
#TODO instead of joining df_gene, exon_info -> directly read in from R feather file & also use that for calculating chrom pos

In [18]:
#IMPORTANT
df_exon = calc_exon_data(path) # exon starts and ends per transcript + exon id
df_exon = df_exon.merge(df_gene[['ensembl_transcript_id','chromosome_name','strand']], left_on = 'transcript_id', right_on = 'ensembl_transcript_id', how='left')
df_exon[['chrom_exon_start','chrom_exon_end']] = df_exon[['chrom_exon_start','chrom_exon_end']].astype('Int64')
del df_exon['transcript_id']
del df_exon['ensembl_transcript_id']
df_exon[['strand']] = df_exon[['strand']].astype('str').astype('category')
df_exon[['exon_id']] = df_exon[['exon_id']].astype('str').astype('category')
df_exon.to_parquet(path/'ensembl_exon_info.parquet')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 819237 entries, 0 to 819236
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   exon_id           819237 non-null  object  
 1   chrom_exon_start  819237 non-null  Int64   
 2   chrom_exon_end    819237 non-null  Int64   
 3   chromosome_name   817389 non-null  category
 4   strand            817389 non-null  category
dtypes: Int64(2), category(2), object(1)
memory usage: 28.9+ MB


In [33]:
#IMPORTANT
df_tran = df_gene[['ensembl_transcript_id','chromosome_name','strand']]
df_tran[['ensembl_transcript_id']] = df_tran[['ensembl_transcript_id']].astype('str').astype('category')
df_tran.to_parquet(path/'ensembl_transcript_info.parquet')

In [ ]:
#read in bs + make subset 
bs = pd.read_parquet(path/'bs_all.parquet') #983499270 rows
bs_new = bs[['miRNA','mRNA','bs_start','bs_end']].copy()
bs_new['miRNA'] = pd.Categorical(bs_new.miRNA)
bs_new['mRNA'] = pd.Categorical(bs_new.mRNA)
bs_new.info()
bs_new.to_parquet(path/'bs_subset.parquet')

In [28]:
bs
#TODO commit
#TODO prepare kidney data as categorical & uint & parquet format

,miRNA,mRNA,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,number_of_pairings,binding_region_length,longest_consecutive_pairings,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_start,bs_end
0,hsa-let-7a-5p,ENST00000476331,1.000000,-18.4,1,1.777982e-03,0.426,1.394372,1.710881,-4.026374,14.0,17.0,9.0,7.0,1.0,6.0,301,340
1,hsa-let-7a-2-3p,ENST00000476331,0.692308,-18.5,0,2.694336e-04,0.456,0.275755,1.061554,-13.549385,16.0,18.0,10.0,1.0,6.0,1.0,423,441
2,hsa-let-7a-2-3p,ENST00000476331,0.769231,-18.6,0,6.177285e-05,0.559,0.251558,0.853162,-16.125569,16.0,23.0,8.0,8.0,7.0,3.0,158,203
3,hsa-let-7b-5p,ENST00000476331,0.615385,-21.7,0,5.099209e-09,0.500,0.233710,-0.052141,-6.514540,19.0,24.0,9.0,15.0,8.0,3.0,11,35
4,hsa-let-7b-5p,ENST00000476331,1.000000,-18.4,1,1.777982e-03,0.426,1.018645,1.724342,-4.026374,14.0,17.0,9.0,7.0,1.0,6.0,318,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983499265,hsa-miR-12125,ENST00000511501,0.538462,-16.1,0,1.360765e-06,0.338,0.000000,0.000000,-8.350718,12.0,16.0,5.0,1.0,0.0,6.0,81,97
983499266,hsa-miR-12126,ENST00000511501,1.000000,-16.3,1,1.420751e-06,0.368,0.000000,0.000000,-11.418273,9.0,10.0,9.0,1.0,0.0,6.0,18,28
983499267,hsa-miR-12127,ENST00000511501,0.923077,-20.6,1,1.328924e-04,0.412,0.000000,0.000000,-7.781775,19.0,24.0,11.0,13.0,5.0,1.0,42,66
983499268,hsa-miR-12129,ENST00000511501,0.538462,-15.7,1,3.936156e-04,0.618,0.000000,0.000000,-9.061245,16.0,21.0,8.0,13.0,5.0,3.0,190,211


In [ ]:
#split bs in 10 parts
for i in range(1,9):
    bs = pd.read_parquet(path/'bs_all.parquet') #983499270 rows
    bs = bs[(i-1)*100000000:i*100000000]
    print('read bs')
    compute_chrom_pos(bs, path/('bs_chrom_pos'+str(i)+'.parquet'), exon_positions)
    print('saved parquet',i)#387.26 GB

read bs


In [23]:
#parse miRNA and mRNA seed from sequence
miRNA_sequences = parse_seq(path_tarp/'input_miRNA.fasta') #2661 miRNAs 
mRNA_sequences = pd.read_parquet(path/'mRNA_all.parquet')
print('All miRNAs and mRNAs were read into pandas.')

bs['miRNA_seed'] = bs.apply(lambda row: miRNA_sequences[row['miRNA']], axis=1)
bs['mRNA_bs_seq'] = bs.apply(lambda row: mRNA_sequences[row['mRNA']][row['bs_start']:row['bs_end']], axis=1)

In [6]:
%load_ext Cython

In [7]:
%%cython
def pos_to_chrom(pos, tid, ids, exon_starts, exon_ends, chrom_exon_starts):
    for j, eid in enumerate(ids[tid]):
        if tid in exon_starts and eid in exon_starts[tid] and tid in exon_ends and eid in exon_ends[tid]:
            if pos >= exon_starts[tid][eid] and pos <= exon_ends[tid][eid]:
                diff = pos - exon_starts[tid][eid]
                chrom_pos = chrom_exon_starts[tid][eid] + diff
                return chrom_pos
        else: print('Error: Either',tid,'or',eid,'not in exon_starts or exon_ends')
    return None

In [8]:
%%cython
def get_eid(start, end, tid, ids, exon_starts, exon_ends, chrom_exon_starts):
    for j, eid in enumerate(ids[tid]):
        if tid in exon_starts and eid in exon_starts[tid] and tid in exon_ends and eid in exon_ends[tid]:
            if (start >= exon_starts[tid][eid]) and (start <= exon_ends[tid][eid]) and (end >= exon_starts[tid][eid]) and (end <= exon_ends[tid][eid]):
                return eid
        else: print('Error: Either',tid,'or',eid,'not in exon_starts or exon_ends')
    return None

In [ ]:
#translate genome position relative to transcript to chromosome
#fastest, always puts bs start and end even if bs spans several exons, but only puts exonid if only 1 exon
bs['chrom_bs_start'] = bs.apply(lambda row: pos_to_chrom(row.bs_start, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
bs['chrom_bs_end'] = bs.apply(lambda row: pos_to_chrom(row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
bs['exon_id'] = bs.apply(lambda row: get_eid(row.bs_start, row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
bs['bs_id'] = bs.index
bs.to_parquet(path/'bs_chrom_pos.parquet')
bs.head()

## Read in GDC data (processed by Xena) from file

In [15]:
#download GDC ChGR37 data from web, once, TCGA Kidney Chromophobe (KICH)
#https://xenabrowser.net/datapages/?hub=https://tcga.xenahubs.net:443
'''
#!wget https://tcga.xenahubs.net/download/unc_v2_exon_hg19_probe_TCGA #unnecessary cause some exons missing
!wget https://tcga.xenahubs.net/download/TCGA.KICH.sampleMap/HiSeqV2_exon.gz
!wget https://tcga.xenahubs.net/download/TCGA.KICH.sampleMap/miRNA_HiSeq_gene.gz
!gunzip HiSeqV2_exon.gz
!gunzip miRNA_HiSeq_gene.gz
!mv HiSeqV2_exon data/KICH
!mv miRNA_HiSeq_gene data/KICH
!mv unc_v2_exon_hg19_probe_TCGA data/KICH'''

'\n#!wget https://tcga.xenahubs.net/download/unc_v2_exon_hg19_probe_TCGA #unnecessary cause some exons missing\n!wget https://tcga.xenahubs.net/download/TCGA.KICH.sampleMap/HiSeqV2_exon.gz\n!wget https://tcga.xenahubs.net/download/TCGA.KICH.sampleMap/miRNA_HiSeq_gene.gz\n!gunzip HiSeqV2_exon.gz\n!gunzip miRNA_HiSeq_gene.gz\n!mv HiSeqV2_exon data/KICH\n!mv miRNA_HiSeq_gene data/KICH\n!mv unc_v2_exon_hg19_probe_TCGA data/KICH'

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def split_chrom_column(chrom):
    chrom_list = chrom.split(':')
    chromosome_name = str(chrom_list[0])#[3:])
    strand = 1 if chrom_list[2] == '+' else -1
    chrom_exon_start = int(chrom_list[1].split('-')[0])
    chrom_exon_end = int(chrom_list[1].split('-')[1])
    return chromosome_name,strand,chrom_exon_start,chrom_exon_end

In [40]:
#IMPORTANT
mirna_counts = pd.read_csv(tcga_path/'miRNA_HiSeq_gene', delimiter='	')

In [43]:
#TODO many values NaN!!!
count_nans = mirna_counts.isnull().sum().sum()
print(count_nans,'values are NaN out of',1917*90,'values (=',(count_nans/(1917*90))*100,'%)')

102885 values are NaN out of 172530 values (= 0.5963310728568945 %)


In [45]:
mirna_counts.isnull().sum(axis=1) #amount of NaNs per miRNA
mirna_counts.isnull().sum() #amount of NaNs per sample

0       86
1       76
2       82
3       78
4       63
        ..
1912    82
1913    63
1914    63
1915    74
1916     2
Length: 1917, dtype: int64

In [50]:
#IMPORTANT
trans = pd.read_feather(path/'mature2families.feather')
trans[['mature_name']] = trans[['mature_name']].astype('category')
mirna_counts = mirna_counts.merge(trans, left_on='sample',right_on='mature_acc', how='left')
del mirna_counts['mature_acc']
mirna_counts.to_parquet(path/'KICH'/'miRNA_counts.parquet')

,mature_acc,mature_name,mirna_family
0,MIMAT0000062,hsa-let-7a-5p,let-7
1,MIMAT0004481,hsa-let-7a-3p,let-7
2,MIMAT0010195,hsa-let-7a-2-3p,let-7
3,MIMAT0000063,hsa-let-7b-5p,let-7
4,MIMAT0004482,hsa-let-7b-3p,let-7


In [60]:
mirna_counts[mirna_counts.mature_name=='hsa-let-7a-5p']

,sample,TCGA-KN-8426-01,TCGA-KN-8419-11,TCGA-KL-8332-01,TCGA-KN-8423-01,TCGA-KO-8405-01,TCGA-KL-8337-01,TCGA-KN-8424-11,TCGA-KN-8422-11,TCGA-KL-8334-01,...,TCGA-KL-8323-01,TCGA-KN-8429-11,TCGA-KO-8413-01,TCGA-KO-8411-01,TCGA-KL-8332-11,TCGA-KL-8335-01,TCGA-KL-8339-11,TCGA-KO-8415-01,mature_name,mirna_family
692,MIMAT0000062,14.422574,14.82746,14.880758,14.793954,14.541135,15.14849,14.417657,14.382524,14.991116,...,15.348145,14.600445,14.474857,14.568523,14.486888,15.229448,14.887372,14.794259,hsa-let-7a-5p,let-7


In [56]:
mirna_counts.count(0).sum() #I cant just replace Nan with 0

73451

In [ ]:
mirna_counts.group

In [ ]:
frame['c'] = frame.a.fillna(0) + frame.b.fillna(0)

In [55]:
mirna_counts.to_parquet(path/'KICH'/'miRNA_counts.parquet')

In [2]:
#IMPORTANT
f = Path(path/'exon_counts_kidney.feather')
if f.is_file():
    print('read from file')
    exon_counts = pd.read_feather(f)
else:
    print('translate exon count positions from GRCh37 to GRCh38')
    exon_counts = pd.read_csv(tcga_path/'HiSeqV2_exon', delimiter='	')
    exon_counts[['chrom_old','strand_old','start_old','end_old']] = exon_counts['sample'].apply(lambda x: pd.Series(split_chrom_column(str(x))))
    #save positions to BED input file
    exon_counts['score'] = 0
    exon_counts.to_csv('data/exon_hg19_pos.bed', sep='\t', columns=['chrom_old','start_old','end_old','sample','score','strand_old'], header=False, index=False)
    #translate positions from GRCh37 to GRCh38
    !../liftOver data/exon_hg19_pos.bed data/hg19ToHg38.over.chain.gz data/exon_hg38_pos.bed data/unlifted.bed
    #read in translated positions from liftOver output BED file
    translated_positions = pd.read_csv('data/exon_hg38_pos.bed', delimiter='	', names=['chromosome_name','chrom_exon_start','chrom_exon_end','id','score','strand'])
    exon_counts = exon_counts.merge(translated_positions[['chromosome_name','chrom_exon_start','chrom_exon_end','id','strand']], left_on='sample', right_on='id', how='inner',validate='1:1')
    exon_counts.drop(['sample','chrom_old','strand_old','start_old','end_old','score'], axis=1, inplace=True)
    #transform strand {-,+} to {-1,+1}
    exon_counts['strand'] = exon_counts.apply(lambda row: -1 if row.strand == '-' else 1, axis=1)
    exon_counts['chromosome_name'] = exon_counts['chromosome_name'].str[3:]
    exon_counts[['chromosome_name','strand']] = exon_counts[['chromosome_name','strand']].astype('str').astype('category')
    exon_counts[['chrom_exon_start','chrom_exon_end']] = exon_counts[['chrom_exon_start','chrom_exon_end']].astype('Int64')

    #TODO option 2: get exonid per exon count from Ensembl (compare)
    #exon_counts2 = exon_counts.merge(df_exon[['exon_id','chromosome_name','strand','chrom_exon_start','chrom_exon_end']], left_on = ['chrom','strand','chromStart','chromEnd'], right_on = ['chromosome_name','strand','chrom_exon_start','chrom_exon_end'], how='left')

    caseids = list(set(mirna_counts.columns).intersection(list(exon_counts.columns)))
    exon_counts.to_feather(f)

read from file


In [22]:
#IMPORTANT 
#TODO merge to check how many exon start + end after translation right / wrong
#alternative: first exon_id then translate
#TODO change slides
#chromosome_name strand chrom_exon_start chrom_exon_end
exon_counts['exon_id'] = exon_counts.merge(df_exon, in sqlite

NameError: name 'ensembl_exons' is not defined

In [26]:
df_exon

,exon_id,chrom_exon_start,chrom_exon_end,chromosome_name,strand
0,ENSE00001544501,577,647,MT,1
1,ENSE00001544499,648,1601,MT,1
2,ENSE00001544498,1602,1670,MT,1
3,ENSE00001544497,1671,3229,MT,1
4,ENSE00002006242,3230,3304,MT,1
...,...,...,...,...,...
819232,ENSE00003815305,128348163,128348438,CHR_HSCHR3_9_CTG2_1,1
819233,ENSE00003827353,128343834,128344022,CHR_HSCHR3_9_CTG2_1,1
819234,ENSE00003819782,128348163,128348718,CHR_HSCHR3_9_CTG2_1,1
819235,ENSE00003835089,128188319,128188643,CHR_HSCHR3_9_CTG2_1,-1


In [ ]:
conn = sqlite3.connect(':memory:')
#write the tables
df_exon.to_sql('df_exon', conn, index=False, dtype={"chrom_exon_start": 'INTEGER', "chrom_exon_end": 'INTEGER'})
exon_counts.to_sql('exon_counts', conn, index=True, index_label='exon_id', dtype={caseid : 'INTEGER' for caseid in caseids}.update({"chrom_exon_start": 'INTEGER', "chrom_exon_end": 'INTEGER'}))
qry = '''
    CREATE TABLE n_counts AS
    select
        *
    from
        exon_counts inner join df_exon
        on
        df_exon.chromosome_name = exon_counts.chromosome_name and df_exon.strand = exon_counts.strand and df_exon.chrom_bs_start >= exon_counts.chrom_exon_start and df_exon.chrom_bs_end <= exon_counts.chrom_exon_end
    '''
pd.read_sql_query(qry,conn)

In [5]:
#IMPORTANT
n_exon_counts = exon_counts.copy()
n_exon_counts = pd.melt(n_exon_counts, id_vars=["chromosome_name", "chrom_exon_start","chrom_exon_end","id","strand"], 
                  var_name="patient_id", value_name="exon_count")
n_exon_counts['patient_id'] = n_exon_counts['patient_id'].astype('category')
del n_exon_counts['id']
n_exon_counts.info() #560 MB

In [7]:
n_exon_counts.to_parquet(path/'KICH'/'exon_counts.parquet') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21730436 entries, 0 to 21730435
Data columns (total 6 columns):
 #   Column            Dtype   
---  ------            -----   
 0   chromosome_name   category
 1   chrom_exon_start  int64   
 2   chrom_exon_end    int64   
 3   strand            category
 4   patient_id        category
 5   exon_count        float64 
dtypes: category(3), float64(1), int64(2)
memory usage: 559.5 MB


In [17]:
exon_counts.info()

,TCGA-KN-8419-01,TCGA-KL-8346-01,TCGA-KN-8422-01,TCGA-KN-8431-11,TCGA-KN-8430-11,TCGA-KM-8440-01,TCGA-KO-8414-01,TCGA-KL-8323-01,TCGA-KM-8639-01,TCGA-KO-8415-11,...,TCGA-KO-8408-01,TCGA-KM-8443-01,TCGA-KM-8442-01,TCGA-KL-8332-11,TCGA-KL-8327-01,chromosome_name,chrom_exon_start,chrom_exon_end,id,strand
0,2.1156,1.7396,2.0641,3.1457,3.0436,2.2209,1.8213,1.9092,2.9139,3.1049,...,2.2369,2.4951,2.1010,3.0617,0.7424,3,51973965,51974630,chr3:52007981-52008646:-,-1
1,0.0000,0.0000,0.0000,0.0251,0.0233,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,1,215728030,215728384,chr1:215901372-215901726:-,-1
2,0.3884,0.2219,0.2050,0.9168,0.6069,0.4547,0.1571,0.3857,0.7535,0.8673,...,0.1602,0.3507,0.7154,0.4353,0.0966,17,51191601,51193016,chr17:49268962-49270377:-,-1
3,2.5290,0.7196,1.3094,2.3074,1.6894,1.5829,1.0835,1.9041,1.2131,1.7214,...,0.6381,1.5227,1.7786,1.4157,0.0000,4,146292929,146292980,chr4:147214081-147214132:-,-1
4,0.0000,0.0000,0.0000,1.4721,0.1284,0.0000,0.0000,0.0000,0.0374,2.4087,...,0.0000,0.0189,0.1334,4.8648,0.0900,1,15747546,15747980,chr1:16074041-16074475:+,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238791,2.4455,5.1078,2.9468,5.4212,5.1299,4.5371,3.5954,4.4245,4.4484,5.5820,...,5.0518,4.9067,3.8206,7.3459,4.1126,3,51978259,51978374,chr3:52012275-52012390:+,1
238792,1.5701,0.5500,0.3451,2.2783,1.7366,0.6813,0.5851,1.4460,2.4188,1.8648,...,0.8498,1.0774,0.8774,1.2410,0.2725,6,43357126,43357765,chr6:43324864-43325503:-,-1
238793,0.4810,0.4219,0.3687,0.7313,0.7666,0.7646,2.4586,0.9557,1.1452,1.9724,...,1.3415,1.8842,0.8646,0.0000,1.9102,17,36168630,36168695,chr17:34495988-34496053:-,-1
238794,2.6944,2.2891,1.8186,4.0291,4.0015,3.7968,2.3984,2.6564,4.2824,3.6833,...,3.0631,1.8938,2.9015,2.3518,1.1468,1,120833602,120833774,chr1:148010884-148011056:-,1


## Map miRNA to counts

In [ ]:
#get subset counts of bs and merge with df_gene
counts = bs[['bs_id', 'miRNA', 'mRNA', 'chrom_bs_start', 'chrom_bs_end']].merge(df_gene[['ensembl_transcript_id','chromosome_name','strand']], left_on='mRNA', right_on='ensembl_transcript_id', how='left')

#mapping mature ID to miRNA family
#trans = pd.read_feather(path/'mature2families.feather')
#counts = counts.merge(trans, left_on='miRNA',right_on='mature_name', how='left')

mirna_counts
trans = pd.read_feather(path/'mature2families.feather')
mirna_counts = mirna_counts.merge(trans, left_on='miRNA',right_on='mature_name', how='left')

#alternative 0: dont join with mirnaids
#counts.drop(['ensembl_transcript_id', 'mature_name'], axis=1, inplace=True)

#! alternative 1: for all caseids at the same time !
counts = counts.merge(mirna_counts, left_on='mature_acc', right_on='sample', how='inner')
counts.drop(['ensembl_transcript_id', 'mature_name', 'sample'], axis=1, inplace=True)

# alternative 2: for 1 caseids at a time
for caseid in []:#[caseids[0]]:
    counts = counts.merge(mirna_counts[['sample',caseid]], left_on='mature_acc', right_on='sample', how='left')
    counts.drop(['ensembl_transcript_id', 'mature_name', 'sample'], axis=1, inplace=True)

#counts[['chromosome_name','strand','miRNA','mRNA']] = counts[['chromosome_name','strand','miRNA','mRNA']].astype('str')#.astype('category')
counts.head()

In [41]:
counts

,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end,chromosome_name,strand,mature_acc,mirna_family,TCGA-KN-8426-01,...,TCGA-KO-8408-01,TCGA-KM-8438-01,TCGA-KL-8323-01,TCGA-KN-8429-11,TCGA-KO-8413-01,TCGA-KO-8411-01,TCGA-KL-8332-11,TCGA-KL-8335-01,TCGA-KL-8339-11,TCGA-KO-8415-01
0,0,hsa-let-7a-2-3p,ENST00000576537,1578440,1578466,17,-1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
1,1,hsa-let-7a-2-3p,ENST00000576537,1578561,1578581,17,-1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
2,9595,hsa-let-7a-2-3p,ENST00000469694,240462246,240462269,2,1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
3,19235,hsa-let-7a-2-3p,ENST00000670658,195454546,195454591,2,1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
4,23207,hsa-let-7a-2-3p,ENST00000474430,155178044,155178063,1,1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52767,61977,hsa-miR-542-3p,ENST00000414717,11446564,11446583,3,1,MIMAT0003389,mir-542,9.029325,...,7.874904,8.344636,7.057697,8.625030,7.208745,7.794783,8.570388,8.508410,9.049981,8.300311
52768,65657,hsa-miR-542-3p,ENST00000570409,3651006,3651031,16,-1,MIMAT0003389,mir-542,9.029325,...,7.874904,8.344636,7.057697,8.625030,7.208745,7.794783,8.570388,8.508410,9.049981,8.300311
52769,62648,hsa-miR-3611,ENST00000414717,11426846,11426866,3,1,MIMAT0017988,hsa-miR-3611,NaN,...,NaN,NaN,NaN,0.170100,NaN,NaN,NaN,0.180300,NaN,NaN
52770,62772,hsa-miR-3910,ENST00000414717,11364682,11364729,3,1,MIMAT0018184,mir-3910,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
#TODO put in presentation, analysis mirna, how many in xena data
#TODO find out : if na should i use it as 0????
counts['TCGA-KN-8426-01'].isna().sum() #for caseid TCGA-KN-8426-01: 56.035 bs are thrown away 
len(list(counts[counts['TCGA-KN-8426-01'].isna()].mature_acc.unique())) #we don't have the counts for 1948 mirnas in bs
len(list(counts[~ counts['TCGA-KN-8426-01'].isna()].mature_acc.unique())) #we have counts for 658 mirnas

len(list(mirna_counts['sample'].unique())) #1917 = absolute nr mirnas from TCGA-KN-8426-01
len(list(mirna_counts[~ mirna_counts['TCGA-KN-8426-01'].isna()]['sample'].unique())) #681 = not na nr mirnas from TCGA-KN-8426-01

1917

## Map exons to counts

In [65]:
#for a large data set, you will likely get a significant speed increase by creating an index for any column(s) used in the join condition. 
#https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others
#instead use SQL
conn = sqlite3.connect('data/db.db')#':memory:') #Make the database in memory
c = conn.cursor()

#write the tables
counts.to_sql('counts', conn, index=False, dtype={"bs_id": 'INTEGER'})
exon_counts.to_sql('exon_counts', conn, index=True, index_label='exon_id', dtype={caseid : 'INTEGER' for caseid in caseids}.update({"chrom_exon_start": 'INTEGER', "chrom_exon_end": 'INTEGER'}))

In [71]:
#print sqlite3 table 
qry = '''
    select exon_counts.chromosome_name from exon_counts
    '''
#c.execute(qry)
#conn.commit()
#c.fetchall()
pd.read_sql_query(qry,conn)

,chromosome_name
0,3
1,1
2,17
3,4
4,1
...,...
238791,3
238792,6
238793,17
238794,1


In [64]:
#DELETE table from sqlite3 DB
c.execute("DROP TABLE counts;")
conn.commit()
c.execute("DROP TABLE exon_counts;")
conn.commit()

In [ ]:
#TODO change column name case_id to column name case_id_mirna so it doesnt overlap with exon
#instead of joining like i do & repeating the exon info -> only add exon_id + pivot / addup per exon + add exon counts

In [ ]:
qry = '''
    CREATE TABLE n_counts AS
    select
        *
    from
        ( 
        select exon_counts.chromosome_name, exon_counts.chrom_exon_start, exon_counts.chrom_exon_end, exon_counts.strand, exon_counts.id
        from exon_counts 
        ) as temp
        inner join
        counts
        on
        counts.chromosome_name = temp.chromosome_name and counts.strand = temp.strand and counts.chrom_bs_start >= temp.chrom_exon_start and counts.chrom_bs_end <= temp.chrom_exon_end
    '''
c.execute(qry)
conn.commit()
#TODO this finds the exon id for every bs 
#52772 -> 18736 bindingsites
#only returns bs with minimum 1 exon_id cause inner join (else left join)
#automatically filters out all binding sites that have more than 1 exon_id
#if u dont want that use outer join -> actually wrong, it uses the first instead of leaving it out!

In [75]:
qry = '''
    select * from n_counts
    '''
n_counts = pd.read_sql_query(qry,conn)

In [76]:
n_counts

,chromosome_name,chrom_exon_start,chrom_exon_end,strand,id,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end,...,TCGA-KO-8408-01,TCGA-KM-8438-01,TCGA-KL-8323-01,TCGA-KN-8429-11,TCGA-KO-8413-01,TCGA-KO-8411-01,TCGA-KL-8332-11,TCGA-KL-8335-01,TCGA-KL-8339-11,TCGA-KO-8415-01
0,1,155178002,155178255,1,chr1:155150478-155150731:+,23204,hsa-let-7a-5p,ENST00000474430,155178054,155178091,...,14.502304,14.302077,15.348145,14.600445,14.474857,14.568523,14.486888,15.229448,14.887372,14.794259
1,1,155178002,155178255,1,chr1:155150478-155150731:+,23207,hsa-let-7a-2-3p,ENST00000474430,155178044,155178063,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
2,1,155178002,155178255,1,chr1:155150478-155150731:+,23219,hsa-let-7c-3p,ENST00000474430,155178044,155178063,...,2.714853,2.455017,2.155998,3.673442,2.676288,3.043974,4.407776,2.500585,3.844001,2.977714
3,1,155178002,155178255,1,chr1:155150478-155150731:+,23229,hsa-let-7e-3p,ENST00000474430,155178044,155178066,...,2.523778,3.836892,2.846723,4.056371,2.882109,3.414638,3.685095,2.885547,3.698584,3.274872
4,1,155178002,155178255,1,chr1:155150478-155150731:+,23230,hsa-let-7f-5p,ENST00000474430,155178054,155178091,...,12.768630,13.254969,13.261290,12.433737,12.189768,12.644354,12.628264,13.546305,12.972790,12.892615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18731,17,1576597,1576720,-1,chr17:1479891-1480014:-,3197,hsa-miR-4433b-5p,ENST00000576537,1576618,1576639,...,NaN,0.214300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18732,17,1576597,1576720,-1,chr17:1479891-1480014:-,3201,hsa-miR-1273h-3p,ENST00000576537,1576685,1576706,...,NaN,NaN,0.194100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18733,17,1576597,1576720,-1,chr17:1479891-1480014:-,3217,hsa-miR-7850-5p,ENST00000576537,1576613,1576660,...,NaN,NaN,0.365156,0.170100,0.316200,0.264600,NaN,NaN,NaN,0.408745
18734,17,1576597,1576720,-1,chr17:1479891-1480014:-,3230,hsa-miR-7976,ENST00000576537,1576619,1576638,...,0.346500,NaN,0.194100,NaN,NaN,NaN,NaN,0.180300,NaN,0.218800


In [86]:
#visualize amount exons by bindingsite | amount bindingsites by exon
#TODO maybe also include other cases eg 1 & 5
qry = '''SELECT * FROM counts inner join (
        select exon_counts.chromosome_name, exon_counts.chrom_exon_start, exon_counts.chrom_exon_end, exon_counts.strand, exon_counts.id
        from exon_counts ) as temp
        on
        counts.chromosome_name = temp.chromosome_name and counts.strand = temp.strand and ((counts.chrom_bs_start >= temp.chrom_exon_start and counts.chrom_bs_start <= temp.chrom_exon_end) or (counts.chrom_bs_end >= temp.chrom_exon_start and counts.chrom_bs_end <= temp.chrom_exon_end))
    '''
bs_exons2 = pd.read_sql_query(qry, conn)
bs_exons2
#bs_exons2.groupby('bs_id').count().raw_counts.hist()
#bs_exons2.groupby('exon_id').count()

,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end,chromosome_name,strand,mature_acc,mirna_family,TCGA-KN-8426-01,...,TCGA-KO-8411-01,TCGA-KL-8332-11,TCGA-KL-8335-01,TCGA-KL-8339-11,TCGA-KO-8415-01,chromosome_name,chrom_exon_start,chrom_exon_end,strand,id
0,9595,hsa-let-7a-2-3p,ENST00000469694,240462246,240462269,2,1,MIMAT0010195,let-7,2.893871,...,1.854876,2.824365,2.395032,2.791374,2.819371,2,240462191,240462582,1,chr2:241401608-241401999:+
1,23207,hsa-let-7a-2-3p,ENST00000474430,155178044,155178063,1,1,MIMAT0010195,let-7,2.893871,...,1.854876,2.824365,2.395032,2.791374,2.819371,1,155178002,155178255,1,chr1:155150478-155150731:+
2,32328,hsa-let-7a-2-3p,ENST00000412954,133435201,133435231,6,-1,MIMAT0010195,let-7,2.893871,...,1.854876,2.824365,2.395032,2.791374,2.819371,6,133435078,133435570,-1,chr6:133756216-133756708:-
3,32329,hsa-let-7a-2-3p,ENST00000412954,133435386,133435423,6,-1,MIMAT0010195,let-7,2.893871,...,1.854876,2.824365,2.395032,2.791374,2.819371,6,133435078,133435570,-1,chr6:133756216-133756708:-
4,39797,hsa-let-7a-2-3p,ENST00000526202,75760745,75760767,13,1,MIMAT0010195,let-7,2.893871,...,1.854876,2.824365,2.395032,2.791374,2.819371,13,75760444,75761038,1,chr13:76334580-76335174:+


In [99]:
#bs_exons2[['chrom_bs_start','chrom_bs_end','chrom_exon_start','chrom_exon_end']][~((bs_exons2.chrom_exon_start<=bs_exons2.chrom_bs_start) & (bs_exons2.chrom_exon_end>=bs_exons2.chrom_bs_end))]

,chrom_bs_start,chrom_bs_end,chrom_exon_start,chrom_exon_end
12,1576707,1578261,1576597,1576720
13,1576707,1578261,1578250,1578323
15,215124742,215169216,215169199,215169359
16,215124742,215169216,215124633,215124750
39,75800869,75804292,75800684,75800882
...,...,...,...,...
24862,75853378,75855273,75853092,75853388
24864,75804538,75817180,75804289,75804541
24865,75804538,75817180,75817161,75817278
24874,75761038,75795419,75760444,75761038


## Pivot tables & put all case_id counts into one

In [104]:
counts.head()

,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end,chromosome_name,strand,mature_acc,mirna_family,TCGA-KN-8426-01,...,TCGA-KO-8408-01,TCGA-KM-8438-01,TCGA-KL-8323-01,TCGA-KN-8429-11,TCGA-KO-8413-01,TCGA-KO-8411-01,TCGA-KL-8332-11,TCGA-KL-8335-01,TCGA-KL-8339-11,TCGA-KO-8415-01
0,0,hsa-let-7a-2-3p,ENST00000576537,1578440,1578466,17,-1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
1,1,hsa-let-7a-2-3p,ENST00000576537,1578561,1578581,17,-1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
2,9595,hsa-let-7a-2-3p,ENST00000469694,240462246,240462269,2,1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
3,19235,hsa-let-7a-2-3p,ENST00000670658,195454546,195454591,2,1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
4,23207,hsa-let-7a-2-3p,ENST00000474430,155178044,155178063,1,1,MIMAT0010195,let-7,2.893871,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371


In [101]:
exon_counts.head()

,TCGA-KN-8419-01,TCGA-KL-8346-01,TCGA-KN-8422-01,TCGA-KN-8431-11,TCGA-KN-8430-11,TCGA-KM-8440-01,TCGA-KO-8414-01,TCGA-KL-8323-01,TCGA-KM-8639-01,TCGA-KO-8415-11,...,TCGA-KO-8408-01,TCGA-KM-8443-01,TCGA-KM-8442-01,TCGA-KL-8332-11,TCGA-KL-8327-01,chromosome_name,chrom_exon_start,chrom_exon_end,id,strand
0,2.1156,1.7396,2.0641,3.1457,3.0436,2.2209,1.8213,1.9092,2.9139,3.1049,...,2.2369,2.4951,2.1010,3.0617,0.7424,3,51973965,51974630,chr3:52007981-52008646:-,-1
1,0.0000,0.0000,0.0000,0.0251,0.0233,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,1,215728030,215728384,chr1:215901372-215901726:-,-1
2,0.3884,0.2219,0.2050,0.9168,0.6069,0.4547,0.1571,0.3857,0.7535,0.8673,...,0.1602,0.3507,0.7154,0.4353,0.0966,17,51191601,51193016,chr17:49268962-49270377:-,-1
3,2.5290,0.7196,1.3094,2.3074,1.6894,1.5829,1.0835,1.9041,1.2131,1.7214,...,0.6381,1.5227,1.7786,1.4157,0.0000,4,146292929,146292980,chr4:147214081-147214132:-,-1
4,0.0000,0.0000,0.0000,1.4721,0.1284,0.0000,0.0000,0.0000,0.0374,2.4087,...,0.0000,0.0189,0.1334,4.8648,0.0900,1,15747546,15747980,chr1:16074041-16074475:+,1


In [ ]:
#TODO idea 1: do seperate tables for all caseids + after pivot put all different caseids sqlite tables together into one
#TODO idea 2: 3D
#TODO debug does this do it right?
#pivot counts (which only has bs & mirna) and afterwards merge with exon counts
#INSERT INTO artists_backup 
#SELECT ArtistId, Name
#FROM artists;

In [136]:
#output: for each case_id for each bs_id: interesting miRNA expression + exon expression
#unstack family counts from 2 columns to several columns (mirna_family=name,mirna_read_count=value)
pivoted = n_counts.copy()
#pivoted = pivoted.pivot_table(values='mirna_read_count', index=['exon_id', 'exon_raw_counts'], columns='mirna_family', aggfunc='sum', fill_value=0)
pivoted = pivoted.pivot_table(index=['id','chromosome_name','chrom_exon_start','chrom_exon_end','strand'], columns='mirna_family', aggfunc='sum', fill_value=0)
pivoted#.head()

TCGA-KL-8323-01  \
mirna_family                                                                      hsa-miR-1229-3p   
id                         chromosome_name chrom_exon_start chrom_exon_end strand                   
chr12:49760688-49761370:+  12              49366905         49367587       1               0.0000   
chr12:49765011-49765073:+  12              49371228         49371290       1               0.0000   
chr12:49835513-49835599:+  12              49441730         49441816       1               0.0000   
chr12:49854543-49854820:+  12              49460760         49461037       1               0.0000   
chr13:76334580-76335174:+  13              75760444         75761038       1               0.0000   
...                                                                                           ...   
chr3:11406133-11406208:+   3               11364659         11364734       1               0.0000   
chr3:11421446-11421526:+   3               11379972         11380052       1               0.0000   
chr3:11468278-11468400:+   3               11426804         11426926       1               0.0000   
chr6:133756216-133756708:- 6               133435078        133435570      -1              0.1941   
chr6:133760466-133760619:- 6               133439328        133439481      -1              0.0000   

                                                                                                   \
mirna_family                                                                      hsa-miR-1229-5p   
id                         chromosome_name chrom_exon_start chrom_exon_end strand                   
chr12:49760688-49761370:+  12              49366905         49367587       1             0.000000   
chr12:49765011-49765073:+  12              49371228         49371290       1             0.365156   
chr12:49835513-49835599:+  12              49441730         49441816       1             0.000000   
chr12:49854543-49854820:+  12              49460760         49461037       1             0.365156   
chr13:76334580-76335174:+  13              75760444         75761038       1             0.000000   
...                                                                                           ...   
chr3:11406133-11406208:+   3               11364659         11364734       1             0.365156   
chr3:11421446-11421526:+   3               11379972         11380052       1             0.000000   
chr3:11468278-11468400:+   3               11426804         11426926       1             0.000000   
chr6:133756216-133756708:- 6               133435078        133435570      -1            0.365156   
chr6:133760466-133760619:- 6               133439328        133439481      -1            0.000000   

                                                                                                   \
mirna_family                                                                      hsa-miR-1234-3p   
id                         chromosome_name chrom_exon_start chrom_exon_end strand                   
chr12:49760688-49761370:+  12              49366905         49367587       1                    0   
chr12:49765011-49765073:+  12              49371228         49371290       1                    0   
chr12:49835513-49835599:+  12              49441730         49441816       1                    0   
chr12:49854543-49854820:+  12              49460760         49461037       1                    0   
chr13:76334580-76335174:+  13              75760444         75761038       1                    0   
...                                                                                           ...   
chr3:11406133-11406208:+   3               11364659         11364734       1                    0   
chr3:11421446-11421526:+   3               11379972         11380052       1                    0   
chr3:11468278-11468400:+   3               11426804         11426926       1                    0   
chr6:133756216-133756708:- 6            

In [119]:
n_counts.head() #18736 rows × 103 columns

,chromosome_name,chrom_exon_start,chrom_exon_end,strand,id,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end,...,TCGA-KO-8408-01,TCGA-KM-8438-01,TCGA-KL-8323-01,TCGA-KN-8429-11,TCGA-KO-8413-01,TCGA-KO-8411-01,TCGA-KL-8332-11,TCGA-KL-8335-01,TCGA-KL-8339-11,TCGA-KO-8415-01
0,1,155178002,155178255,1,chr1:155150478-155150731:+,23204,hsa-let-7a-5p,ENST00000474430,155178054,155178091,...,14.502304,14.302077,15.348145,14.600445,14.474857,14.568523,14.486888,15.229448,14.887372,14.794259
1,1,155178002,155178255,1,chr1:155150478-155150731:+,23207,hsa-let-7a-2-3p,ENST00000474430,155178044,155178063,...,2.829471,2.015161,2.594806,2.644861,1.786991,1.854876,2.824365,2.395032,2.791374,2.819371
2,1,155178002,155178255,1,chr1:155150478-155150731:+,23219,hsa-let-7c-3p,ENST00000474430,155178044,155178063,...,2.714853,2.455017,2.155998,3.673442,2.676288,3.043974,4.407776,2.500585,3.844001,2.977714
3,1,155178002,155178255,1,chr1:155150478-155150731:+,23229,hsa-let-7e-3p,ENST00000474430,155178044,155178066,...,2.523778,3.836892,2.846723,4.056371,2.882109,3.414638,3.685095,2.885547,3.698584,3.274872
4,1,155178002,155178255,1,chr1:155150478-155150731:+,23230,hsa-let-7f-5p,ENST00000474430,155178054,155178091,...,12.768630,13.254969,13.261290,12.433737,12.189768,12.644354,12.628264,13.546305,12.972790,12.892615


In [159]:
case_data

,id,TCGA-KL-8323-01,hsa-miR-1229-3p,hsa-miR-1229-5p,hsa-miR-1234-3p,hsa-miR-1268b,hsa-miR-1269b,hsa-miR-1273h-3p,hsa-miR-1295b-3p,hsa-miR-1295b-5p,...,mir-934,mir-935,mir-937,mir-938,mir-939,mir-940,mir-942,mir-943,mir-95,mir-96
0,chr1:155150478-155150731:+,0.0000,0.1941,0.000000,0,0.0000,0,0.0000,0,0,...,0.0000,0,0,0,0.000000,0.365156,4.243361,0,0.000000,0.000000
1,chr6:133760466-133760619:-,0.0000,0.0000,0.000000,0,0.0000,0,0.0000,0,0,...,0.0000,0,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000
2,chr2:241404033-241404163:+,4.8354,0.0000,0.000000,0,0.0000,0,0.0000,0,0,...,0.0000,0,0,0,0.000000,0.000000,4.049261,0,0.000000,0.000000
3,chr22:38878501-38879444:+,2.6066,0.0000,0.365156,0,0.5181,0,0.0000,0,0,...,0.1941,0,0,0,0.365156,0.000000,0.000000,0,0.000000,4.451873
4,chr17:1481544-1481617:-,6.8482,0.0000,0.000000,0,0.0000,0,0.0000,0,0,...,0.0000,0,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,chr13:76334580-76335174:+,1.7193,0.0000,0.000000,0,0.0000,0,0.0000,0,0,...,0.0000,0,0,0,0.000000,0.000000,0.000000,0,1.005966,0.000000
59,chr1:179335539-179335741:+,0.0000,0.0000,0.365156,0,0.5181,0,0.0000,0,0,...,0.1941,0,0,0,0.000000,0.000000,0.000000,0,0.000000,4.451873
60,chr12:49854543-49854820:+,2.7075,0.0000,0.365156,0,0.5181,0,0.0000,0,0,...,0.0000,0,0,0,0.365156,0.000000,0.000000,0,0.000000,0.000000
61,chr13:76369537-76369567:+,4.6895,0.0000,0.000000,0,0.0000,0,0.0000,0,0,...,0.0000,0,0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000


In [188]:
caseid = 'TCGA-KL-8323-01'
case_data = exon_counts[['id',caseid]].merge(pivoted[caseid], how='inner', left_on='id', right_on='id')
y = np.array(case_data[caseid])
mirnas = trans.mirna_family.drop_duplicates().values # 1868 / 2656
for mirna in mirnas:
    if mirna not in case_data:
        case_data[mirna] = 0
X = case_data[mirnas].values
#TODO check whether there are some mirnas in GDC data which we end up not using as I have no entry in family dictionary for them -> add

[[51.60572298  6.61539042 18.14517162 ...  0.          0.
   0.        ]
 [38.6407133   0.          0.         ...  3.44881894  0.
   0.        ]
 [ 4.74323022  2.20186765  0.         ...  0.          0.
   0.        ]
 ...
 [38.51844668  1.10582756 15.23751325 ...  3.44881894  0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [22.92590321  5.69809075 29.2371607  ...  0.          0.
   0.        ]] ['let-7' 'mir-15' 'mir-17' ... 'mir-1270' 'mir-3118' 'mir-5701']


In [ ]:
#visualize amount exons by bindingsite
#bs_exons.groupby('bs_id').count().raw_counts.hist()

In [190]:
#TODO check - is nr mirna downloaded also here? if not: mistake
print(X.shape,y.shape)

(63, 1868) (63,)


# Elastic Net Regression

In [ ]:
import sys
sys.getsizeof(pivoted)#TODO

In [192]:
#regression btw miRNA expression, exon expression
import sklearn
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
#faster if np.array(x, order='F')
#If you are interested in controlling the L1 and L2 penalty separately, keep in mind that this is equivalent to:
#a * L1 + b * L2 where: alpha = a + b and l1_ratio = a / (a + b)

regr = ElasticNet(random_state=0)
regr.fit(X, y)
print(regr.coef_)
print(regr.intercept_)

[-0.01084431 -0.00921378  0.01006737 ... -0.          0.
  0.        ]
3.436325606218843
